# Connect to Database

In [1]:
# import basic module
import pandas as pd
import os, sys
import warnings
warnings.filterwarnings('ignore')

# get working directory
sys.path.append(os.path.abspath('..'))

import scripts.database_setup as db

**Connection with PostgreSQL** 

In [2]:
engine = db.get_database_connection()

2025-02-02 13:41:41,427 - INFO - Successfully connected to the PostgreSQL.


**Create table in PostgreSQL**

In [3]:
db.create_table(engine)

2025-02-02 13:41:41,435 - INFO - Table 'Telegram_Medical_Message' created succussfully


**Insert data into database**

In [4]:
# load processed data 
processed_df = pd.read_csv('../data/preprocessed_telegram_data.csv', parse_dates=['message_date'])
processed_df.head()

,channel_title,channel_username,message_id,message,message_date,emoji_used,youtube_links
0,CheMed,@CheMed123,97,"️Notice! Dear esteemed customers, Due to four-...",2023-02-10 12:23:06+00:00,⚠🔅🔅,No YouTube Link
1,CheMed,@CheMed123,96,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...,2023-02-02 08:58:52+00:00,📌,No YouTube Link
2,CheMed,@CheMed123,95,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...,2023-02-01 08:59:37+00:00,📌,No YouTube Link
3,CheMed,@CheMed123,94,Che-Med Trivia #3 ምግብና መጠጦች አንዳንድ መድሃኒቶች በደንብ ...,2023-01-31 09:19:53+00:00,No Emoji,No YouTube Link
4,CheMed,@CheMed123,93,"Che-Med Trivia #2 እንደ Ciprofloxacin, Doxycycli...",2023-01-30 09:45:25+00:00,No Emoji,No YouTube Link


In [5]:
# check for missing value
processed_df.isnull().sum()

channel_title        0
channel_username     0
message_id           0
message             35
message_date         0
emoji_used           0
youtube_links        0
dtype: int64

In [6]:
# drop missing Value in messages
processed_df.dropna(subset=['message'], inplace=True)
processed_df.isnull().sum()

channel_title       0
channel_username    0
message_id          0
message             0
message_date        0
emoji_used          0
youtube_links       0
dtype: int64

In [7]:
# Inser Cleaned datan into database
db.insert_data(engine, processed_df)

2025-02-02 13:41:42,101 - INFO - 1781 records inserted into PostgreSQL database.


## Verify inserted data

In [10]:
query = "SELECT * FROM telegram_medical_message LIMIT 5"
pg_df = pd.read_sql(query, engine)
pg_df

,id,channel_title,channel_username,message_id,message,message_date,emoji_used,youtube_links
0,1,CheMed,@CheMed123,97,"️Notice! Dear esteemed customers, Due to four-...",2023-02-10 12:23:06,⚠🔅🔅,No YouTube Link
1,2,CheMed,@CheMed123,96,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...,2023-02-02 08:58:52,📌,No YouTube Link
2,3,CheMed,@CheMed123,95,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...,2023-02-01 08:59:37,📌,No YouTube Link
3,4,CheMed,@CheMed123,94,Che-Med Trivia #3 ምግብና መጠጦች አንዳንድ መድሃኒቶች በደንብ ...,2023-01-31 09:19:53,No Emoji,No YouTube Link
4,5,CheMed,@CheMed123,93,"Che-Med Trivia #2 እንደ Ciprofloxacin, Doxycycli...",2023-01-30 09:45:25,No Emoji,No YouTube Link
